In [25]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn

In [26]:
#Flag for processing training csv or test csv
train = True

In [27]:
if train:
    df = pd.read_csv("train.csv")
else:
    df = pd.read_csv("test_public.csv")

#Remove rows with missing data
df = df[df['MISSING_DATA'] == False]

In [28]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [29]:
len(df)

320

In [30]:
from datetime import datetime
def parse_time(x):
    # We are using python's builtin datetime library
    # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

    # Each x is essentially a 1 row, 1 column pandas Series
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Over every single 
def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
if train:
    df["LEN"] = df["POLYLINE"].apply(polyline_to_trip_duration)
df[["YR", "MON", "DAY", "HR", "WK"]] = df[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [31]:
#Remove all LEN == 0
if train:
    zero_count = len(df[df["LEN"] <= 60])
    print(f"Removing {zero_count} LEN < 60 datapoints")
    df = df[df["LEN"] > 60]

In [32]:
if train:
    #remove outliers
    outlier_threshold = 3
    mean, std = df["LEN"].mean(), df["LEN"].std()
    median = df["LEN"].median()
    df = df[df["LEN"] < mean + outlier_threshold * std]

In [33]:
cols = ["ORIGIN_CALL", "TAXI_ID", "ORIGIN_STAND", "WK", "MON", "HR"]

if train:
    for col in cols:
        items = df[col].unique()

        for t in items:
            mean = df[df[col] == t]["LEN"].mean()
            df.loc[df[col] == t, col + "_AVG_TIME"] = mean
else:
    df_train = pd.read_csv("average_processed_train.csv")
    
    for col in cols:
        items = df[col].unique()
        df_train_nan = df_train[pd.isna(df_train[col])]
        if len(df_train_nan):
            nan_mean = df_train_nan["LEN"].sum() / len(df_train_nan)
        else: 
            nan_mean = 687.1136550505105 #global mean
        
        for t in items:
            mean = df_train[df_train[col] == t]["LEN"].mean()
            df.loc[df[col] == t, col + "_AVG_TIME"] = mean if mean > 0 else nan_mean

In [34]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,YR,MON,DAY,HR,WK,ORIGIN_CALL_AVG_TIME,TAXI_ID_AVG_TIME,ORIGIN_STAND_AVG_TIME,WK_AVG_TIME,MON_AVG_TIME,HR_AVG_TIME
0,T1,B,NaN,15.0,20000542,1408039037,A,False,2014,8,14,17,3,NaN,702.514286,766.610383,710.283479,649.472439,822.471678
1,T2,B,NaN,57.0,20000108,1408038611,A,False,2014,8,14,17,3,NaN,680.091586,659.724281,710.283479,649.472439,822.471678
2,T3,B,NaN,15.0,20000370,1408038568,A,False,2014,8,14,17,3,NaN,622.094241,766.610383,710.283479,649.472439,822.471678
3,T4,B,NaN,53.0,20000492,1408039090,A,False,2014,8,14,17,3,NaN,623.333807,627.823529,710.283479,649.472439,822.471678
4,T5,B,NaN,18.0,20000621,1408039177,A,False,2014,8,14,17,3,NaN,624.266297,718.371884,710.283479,649.472439,822.471678


In [35]:
if train:
    df.to_csv("average_processed_train.csv")

In [36]:
#fast load COMMENT OUT IF NOT FAST LOADING
#import numpy as np, pandas as pd
#import matplotlib.pyplot as plt
#import torch.nn as nn

#train = True

#df = pd.read_csv("average_processed_train.csv")
#df = df.drop(columns = ["Unnamed: 0"])

In [37]:
#df["ORIGIN_STAND"] = df["ORIGIN_STAND"].fillna(0)
#df["ORIGIN_CALL"] = df["ORIGIN_CALL"].fillna(0)

In [38]:
#One hot encoding functions
def encode_a(val):
    return int(val == 'A')

def encode_b(val):
    return int(val == 'B')

def encode_c(val):
    return int(val == 'C')

def encode_year(val):
    return int(val == 2014)

In [39]:
#Actual one hot encoding
df["CALL_TYPE_A"] = df["CALL_TYPE"].apply(encode_a)
df["CALL_TYPE_B"] = df["CALL_TYPE"].apply(encode_b)
df["CALL_TYPE_C"] = df["CALL_TYPE"].apply(encode_c)

In [40]:
df_train = pd.read_csv("average_processed_train.csv")

cols = ["ORIGIN_CALL", "TAXI_ID", "ORIGIN_STAND", "WK", "MON", "HR"]

for col in cols:
    df_train_nan = df_train[pd.isna(df_train[col])]
    if len(df_train_nan):
        nan_mean = df_train_nan["LEN"].sum() / len(df_train_nan)
    else: 
        nan_mean = 687.1136550505105 #global mean
        
    df.loc[pd.isna(df[col]), col + "_AVG_TIME"] = nan_mean
    
#    col_avg = col + "_AVG_TIME"
#    df[col_avg] = (df[col_avg] - df[col_avg].mean()) / df[col_avg].std()

In [41]:
len(df)

320

In [42]:
taxi_ids = list(sorted(df.TAXI_ID.unique()))

In [43]:
df["TAXI_ID"] = df["TAXI_ID"].apply(taxi_ids.index)

In [44]:
#Clean out unnecessary columns
df = df.drop(columns = ["TIMESTAMP", "MISSING_DATA", "CALL_TYPE", "DAY_TYPE", "YR", "ORIGIN_CALL"])
if train:
    df = df.drop(columns = ["POLYLINE", "TRIP_ID"])

In [45]:
df["ORIGIN_STAND"] = df["ORIGIN_STAND"].fillna(0)

In [46]:
df.head()

,TRIP_ID,ORIGIN_STAND,TAXI_ID,MON,DAY,HR,WK,ORIGIN_CALL_AVG_TIME,TAXI_ID_AVG_TIME,ORIGIN_STAND_AVG_TIME,WK_AVG_TIME,MON_AVG_TIME,HR_AVG_TIME,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C
0,T1,15.0,190,8,14,17,3,683.450173,702.514286,766.610383,710.283479,649.472439,822.471678,0,1,0
1,T2,57.0,37,8,14,17,3,683.450173,680.091586,659.724281,710.283479,649.472439,822.471678,0,1,0
2,T3,15.0,129,8,14,17,3,683.450173,622.094241,766.610383,710.283479,649.472439,822.471678,0,1,0
3,T4,53.0,171,8,14,17,3,683.450173,623.333807,627.823529,710.283479,649.472439,822.471678,0,1,0
4,T5,18.0,217,8,14,17,3,683.450173,624.266297,718.371884,710.283479,649.472439,822.471678,0,1,0


In [47]:
len(df)

320

In [48]:
if train:
    df.to_csv("processed_train.csv")
else:
    df.to_csv("processed_test.csv")